In [13]:
from google.colab import drive
drive.mount("/content/drive")
#!cd /content/drive/My Drive/
#drive.mount('/content/drive/My Drive/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from time import time
import math
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Embedding, LSTM, Dense, Dot, Reshape, Lambda, GRU, concatenate, Conv1D, GlobalMaxPool1D, Dropout
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
from keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
from gensim.models import word2vec
from gensim.models import KeyedVectors
import numpy as np
import itertools
import re
%matplotlib inline

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Install Magnitude on Google Colab
! echo "Installing Magnitude.... (please wait, can take a while)"
! (curl https://raw.githubusercontent.com/plasticityai/magnitude/master/install-colab.sh | /bin/bash 1>/dev/null 2>/dev/null)
! echo "Done installing Magnitude."

Installing Magnitude.... (please wait, can take a while)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   137  100   137    0     0   1256      0 --:--:-- --:--:-- --:--:--  1256
Done installing Magnitude.


In [ ]:
from pymagnitude import *
elmo = Magnitude('./drive/MyDrive/Colab Notebooks/dataset/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.magnitude')

In [26]:
def get_vector(text):
    #sentence  = elmo.query(text)
    #unrolled = elmo.unroll(sentence)
    #vectors = np.zeros_like(unrolled[0])
    #for i in range(len(unrolled)):
    #    vectors += unrolled[i]
    #v = np.zeros_like(vectors)
    #for i in range(len(vectors)):
    #    v += vectors[i]
    #return v / len(text)
    sum_vec = np.zeros_like(elmo.query(text[0]))
    for w in text:
        sum_vec += elmo.query(w)
    return sum_vec / len(text)

def sum_qd(q, d):
    sum = 0
    for i in range(len(q)):
        sum += q[i] * d[i]
    return sum

def root_square_sum(v):
    sum = 0
    for i in range(len(v)):
        sum += v[i] * v[i]
    root = math.sqrt(sum)
    return root

def cos_sim(v1, v2):
    if (np.linalg.norm(v1) * np.linalg.norm(v2)) == 0:
        print("no word")
        return 0
    a = sum_qd(v1, v2)
    b = root_square_sum(v1) * root_square_sum(v2)
    return a/b

In [ ]:
# File paths
TRAIN_CSV = './drive/MyDrive/Colab Notebooks/dataset/questions.csv'
# Load training set
train_df = pd.read_csv(TRAIN_CSV, encoding="cp932")
input_left = train_df['question1'].values
input_right = train_df['question2'].values

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
len(input_left)

404351

In [27]:
output_list = []
count = 0
for i in range(len(input_left)):
    doc_w = []
    doc_w.append(str(input_left[i]).split())
    doc_w.append(str(input_right[i]).split())
    #for i in range(2):
    #    words = []
    #    for w in doc_w[i]:
    #        w = w.replace("'s", '')
    #        w = re.sub(re.compile("[!-/:-@[-`{-~]"), '', w)
    #        if w not in stop_words:
    #            if w != '':
    #                words.append(w)
    #    doc_w[i] = words
    v1 = get_vector(doc_w[0])
    v2 = get_vector(doc_w[1])
    output_list.append(cos_sim(v1, v2))
    count += 1
    if count%10000 == 0:
      print(count)
print("Done.")

10000
20000
30000


KeyboardInterrupt: ignored

In [29]:
len(output_list)

37357

In [ ]:
#書き込み
#出力ファイルに文字コードをUTF-8にして出力
with open("./drive/MyDrive/Colab Notebooks/test_data/STS.output_elmo.images.txt", "w", encoding="utf-8") as fw:
    for i in range(len(output_list)):
        print(output_list[i], file=fw)